# Importing Libraries

In [1]:
import torch
#from torch import nn
from torchvision import datasets, transforms

#import numpy as np

#from sklearn import datasets

import matplotlib as mpl
#from matplotlib.colors import LinearSegmentedColormap
#from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt

#import reprlib
import sys

# Configuring Visualization Parameters

In [2]:
%matplotlib inline

In [3]:
XINHUI = "#7a7374"
XUEBAI = "#fffef9"
YINBAI = "#f1f0ed"
YINHUI = "#918072"

figure_size = (16, 9)

In [4]:
custom_params = {
    "axes.axisbelow": True,
    "axes.edgecolor": YINBAI,
    "axes.facecolor": XUEBAI,
    "axes.grid": True,
    "axes.labelcolor": XINHUI,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.titlecolor": XINHUI,
    "figure.edgecolor": YINBAI,
    "figure.facecolor": XUEBAI,
    "grid.alpha": .8,
    "grid.color": YINBAI,
    "grid.linestyle": "--",
    "grid.linewidth": 1.2,
    "legend.edgecolor": YINHUI,
    "patch.edgecolor": XUEBAI,
    "patch.force_edgecolor": True,
    "text.color": XINHUI,
    "xtick.color": YINHUI,
    "ytick.color": YINHUI,
}

mpl.rcParams.update(custom_params)

# Configuring Other Notebook Parameters

In [5]:
#reprlib_rules = reprlib.Repr()
#reprlib_rules.maxother = 250

# Pre-installing Custom Functions

In [6]:
sys.path.append("../")

In [7]:
from Modules import *

# Practicing in Stages

## Image Transforms

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

training_dataset = datasets.MNIST(root="../Datasets",
                                  train=True,
                                  download=True,
                                  transform=transform)

tabulation = Form_Generator()
font_colors_list = tabulation.color_selector()

tabulation.heading_printer("Loading and transformation of the MNIST dataset")

statements = [
    """
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

training_dataset = datasets.MNIST(root="../Datasets",
                                  train=True,
                                  download=True,
                                  transform=transform)
"""
]
tabulation.statement_generator(statements)

variables = ["transform", "training_dataset"]
values = [str(transform), str(training_dataset)]
tabulation.variable_generator(variables, values)

Loading and transformation of the MNIST dataset

    +--------------------------------------------------------+
    | Statement                                              |
    +--------------------------------------------------------+
    | transform = transforms.Compose(                        |
    |     [transforms.ToTensor(),                            |
    |      transforms.Normalize((0.5, ), (0.5, ))])          |
    |                                                        |
    | training_dataset = datasets.MNIST(root="../Datasets",  |
    |                                   train=True,          |
    |                                   download=True,       |
    |                                   transform=transform) |
    +--------------------------------------------------------+
    +------------------+----------------------------------------+
    | Variable         | Value                                  |
    +------------------+---------------------------------------

In [17]:
training_loader = torch.utils.data.DataLoader(dataset=training_dataset,
                                              batch_size=100,
                                              shuffle=True)

tabulation = Form_Generator()
tabulation.heading_printer("Training batch settings")

statements = [
    """
training_loader = torch.utils.data.DataLoader(dataset=training_dataset,
                                              batch_size=100,
                                              shuffle=True)
"""
]
tabulation.statement_generator(statements)

expressions = [
    "len(training_loader)", "list(training_loader)[0][0].shape",
    "list(training_loader)[-1][0].shape"
]
results = [
    str(len(training_loader)),
    str(list(training_loader)[0][0].shape),
    str(list(training_loader)[-1][0].shape)
]
tabulation.expression_generator(expressions, results, 12)

Training batch settings

    +-------------------------------------------------------------+
    | Statement                                                   |
    +-------------------------------------------------------------+
    | training_loader =                                           |
    |     torch.utils.data.DataLoader(dataset=training_dataset,   |
    |                                                             |
    |     batch_size=100,                                         |
    |                                               shuffle=True) |
    +-------------------------------------------------------------+
    +------------------------------------+----------------------+
    | Expression                         | Result               |
    +------------------------------------+----------------------+
    | len(training_loader)               | 600                  |
    | list(training_loader)[0][0].shape  | torch.Size([100, 1,  |
    |                            

In [27]:
expressions = [
    "len(training_loader)", "list(training_loader)[0][0].shape",
    "list(training_loader)[-1][0].shape"
]
results = [
    str(len(training_loader)),
    str(list(training_loader)[0][0].shape),
    "ssssssssssssssssssssss sssssssssssssssssssss"
]
tabulation.expression_generator(expressions, results, 1)

    +------------------------------------+------------------------+
    | Expression                         | Result                 |
    +------------------------------------+------------------------+
    | len(training_loader)               | 600                    |
    | list(training_loader)[0][0].shape  | torch.Size([100, 1,    |
    |                                    |  28, 28])              |
    | list(training_loader)[-1][0].shape | ssssssssssssssssssssss |
    |                                    |  sssssssssssssssssssss |
    +------------------------------------+------------------------+


In [11]:
str(list(enumerate(training_loader))[-1][1][0].shape).split(" ")

['torch.Size([100,', '1,', '28,', '28])']

In [12]:
training_loader[0]

TypeError: 'DataLoader' object is not subscriptable

In [ ]:
list(training_loader)[0][0].shape

In [ ]:
len("torch.Size([100,")

In [ ]:
len("-------------------------+")

In [ ]:
len("+-------------------------------------------------------------+")